In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import parser
import numpy as np

In [2]:
#new_df = ['10205614','90385767', '80385763', '50385750', '60385764', '20385761', '60385759', '80385744','80385758', '10385766', '00385757', '10385752', '30385765', '70385754', '90385753', '40385755', '70385749']
url='../My_data_science/Book1.xls'
data = pd.read_excel(url,sheet_name = 'Sheet1')
df = data['Data'].astype(str)
new_df =[]
# If lenght of article (from excell file) is less then 8 symbols - add two zeroes to article
for art in df:
    if len(art)<8:
        art = "00" + art
        new_df.append(art)
    else:
        new_df.append(art)

'url=\'../My_data_science/Book1.xls\'\ndata = pd.read_excel(url,sheet_name = \'Sheet1\')\ndf = data[\'Data\'].astype(str)\nnew_df =[]\nfor art in df:\n    if len(art)<8:\n        art = "00" + art\n        new_df.append(art)\n    else:\n        new_df.append(art)\nnew_df = new_df'

In [3]:
#Create lists for data collection from web pages

Article_num = []
Article_family = []
Article_part = []
Article_size = []
Article_color = []
Article_quantity1 = []
Article_quantity = []
Article_product_area = []

#Iterate and parce through necessary data on web page


for item in new_df:
    # Here Item is number of article, and necessary web page is changing by changing item_no in hyperlink
    url = 'http://iwww.pia-facts.ikea.com/Pages/PIAFacts.aspx?item_no='+item+'&lang_code=gb&cty_code=OF&ParentItem=&ItemAdd=0'
    html = requests.get(url).content
    df_list = pd.read_html(html)
    df_head = df_list[2]
    df_tail = df_list[-13]
    
    #Creating lists of parced data and imputing specific data there
    Article_num.append(item)
    Article_family.append(df_head[0][0])
    Article_part.append(df_head[1][0])
    Article_size.append(df_head[2][0])
    Article_color.append(df_head[4][0])
    Article_quantity.append(df_head[5][0])
    #Some pages have issues with making, so some of them located in other locations
    try:
        Article_product_area.append(df_tail[1][2])
    except:
        Article_product_area.append(df_list[-14][1][2])
            
Article_size = list(map(lambda x: x[2:],Article_size))
Article_color = list(map(lambda x: x[2:],Article_color))
Article_part = list(map(lambda x: x[2:],Article_part))
#Some articles, in terms of quantity will have no data, because product have only 1 pack, that's why I put string "1 pack" if nan
Np_q = np.array(Article_quantity)
for x in Np_q:
    if x != 'nan':
        Article_quantity1.append(x[2:])
    else:
        Article_quantity1.append('1 pack') 

In [4]:
#Create dict and put inside all parced data
dict_table = {'Article_num':Article_num,'Article_family':Article_family,'Article_part':Article_part,'Size':Article_size,'Color':Article_color,'Quantity':Article_quantity1,'Product_area':Article_product_area}
df = pd.DataFrame(dict_table)  

In [6]:
writer = pd.ExcelWriter('output.xlsx')
df.to_excel(writer,'Sheet1',index = False)
writer.save()
